# Scott Schulze's second code review (I think)?
# HAS Tools Homework 13
# 11/21/20



## Introduction 
Welcome Laura, 
This is easily THE MOST stressful assignment of the semester so far, only because now do I have to do everything we have done, but do it all neatly, and in one place! AHHH! I understand that this assignment should be comforting in that we can all demonstrate all we have learned throughout the semester, but, if you'll pardon the desert pun, in my case it is tapping a dry well. One required item I am fairly certain that you will not see is a map, I never got my head around that after all the issues.
The purpose is to demonstrate many of the topics we've covered in a uniform, easy to ready story-board like presentation. Hopefully by the end, there will have been a function used(we're still not friends, functions and I), two different kinds of plots (they may well be the same data, just viewed two different ways), my forecasts for weeeks 1, 2 and the 16 week. This all will be headed by the discussion immediately following this of where I stand currently and how the forecasting has been going to date.

## How it is made: Forecasting episode
When the semester started out, week 1 especially I made my forecasts just from my own guessing. I am good at recognizing patterns and adjusting. Through the middle part of the course, I used my model for generating the forecasts this was very effective when flows were so low. Now that they have returned to near normal levels, I am back to forecasting from my knowledge as I don't have the wherewithall to change my code to better suit the current conditions.

### Let's start with some libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import urllib.request as req
import urllib
from sklearn.linear_model import LinearRegression
import datetime
import os

###  Now the function
    This function is given the previous week's flow value as an input
    Then uses the previously calculated model intercept and coefficient
    to find the first week's prediction. Then uses the first prediction to
    make the predicition for the second week's forecast. Before returning
    both forecast values. Reverted to a 2-week forecast on 10/24/2020.

    Variables:
    previous_flow: previous_flow is given to the function as the average flow
    from the previous week and is the initialization point for
    prediction: prediction is an array, set to the size of the forecasts being
    generated, the outputs from the model are stored here.

In [ ]:
def predictions(previous_flow):
 
    prediction = np.zeros(2)
    for i in range(len(prediction)):
        if i == 0:
            prediction[i] = model.intercept_ + model.coef_ * previous_flow
        elif i == 1:
            prediction[i] = model.intercept_ + model.coef_ * prediction[i-1]
    return prediction

### Next the data to work with, brought in via API
The first dataset will be from a weather observing station near to the streamgauge, the second will be the streamgage data to be used.

In [ ]:
mytoken = '8a9ebdf8eaba4686b060300750c1658a'
base_url = "http://api.mesowest.net/v2/stations/timeseries"
args = {
    'start': '199701010000',              
    'end': '202011210000',                
    'obtimezone': 'UTC',
    'vars': 'air_temp',
    'stids': 'QVDA3',
    'units': 'temp|F',
    'token': mytoken}
apiString = urllib.parse.urlencode(args)
print(apiString)
fullUrl = base_url + '?' + apiString
print(fullUrl)
response = req.urlopen(fullUrl)
responseDict = json.loads(response.read())


### Now put it in a more useful format for us
The following block takes the imported data and puts it into a pandas dataframe for us to utilize much more efficiently.

In [ ]:
data2 = pd.DataFrame({'Temperature': airT}, index=pd.to_datetime(dateTime))
data_weekly = data2.resample('W').mean().round(1)
print(len(data_weekly))
lwt = data_weekly.tail(1).round(1)
print("Last week's average temperature was ", lwt, "°F.")

### Now bring in the flow data in a similar manner
This section brings in streamgage data from the USGS and formats it, puts it into a dataframe and standardizes by using the datetime function.

In [ ]:
site = '09506000'
start = '1990-01-01'
end = '2020-11-21'      # Update this on Sunday
url = "https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=" \
    + site + "&referred_module=sw&period=&begin_date=" + start \
    + "&end_date=" + end
data = pd.read_table(url, skiprows=30, names=['agency_cd', 'site_no',
                                              'datetime', 'flow', 'code'],
                     parse_dates=['datetime'], index_col='datetime')
data['year'] = pd.DatetimeIndex(data.index).year
data['month'] = pd.DatetimeIndex(data.index).month
data['day'] = pd.DatetimeIndex(data.index).dayofweek
data['dayofweek'] = pd.DatetimeIndex(data.index).dayofweek
flow_weekly = data.resample("W").mean().round(2)
flow_weekly.info()
lwf = flow_weekly[['flow']].tail(1)
